In [ ]:
import logging
from logging.config import dictConfig

dictConfig({
    'version': 1,
    'formatters': {
        'default': {
            'format': '[%(asctime)s] %(message)s',
        }
    },
    'handlers': {
        'file': {
            'level': 'DEBUG',
            'class': 'logging.FileHandler',
            'filename': 'debug.log',
            'formatter': 'default',
        },
    },
    'root': {
        'level': 'DEBUG',
        'handlers': ['file']
    }
})


def myfunc():
    logging.debug("함수가 시작되었습니다.")


myfunc()

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from gluonnlp.data import SentencepieceTokenizer

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 특정 파일 불러오기
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/train_preprocessed_ver2_no_lower_no_ques.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/test/test_preprocessed_ver2_no_lower_no_ques.csv")

In [ ]:
import random
random_seed = 0

torch.manual_seed(random_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
# 중복제거
train_preprocessed["document"] = train_preprocessed["document"].apply(lambda x : x.strip())

Over Sampling

In [ ]:
small_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 20].index
mask = train_preprocessed['label'].isin(small_label)
smalls = train_preprocessed[mask].reset_index(drop=True)
train_preprocessed=train_preprocessed.append(smalls) # 두배
train_preprocessed=train_preprocessed.append(smalls) # 세배
train_preprocessed.reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_preprocessed["label"])

label = le.transform(train_preprocessed["label"])
train_preprocessed["encoded_label"] = label

In [ ]:
# 라벨개수
print(len(le.classes_))

In [ ]:
# dataframe 형태의 데이터를 리스트 형태로 변경
# bert 모델을 위한 데이터
def transform_df_lst(df):
    data_lst = []
    for q, label in zip(df['document'], df['encoded_label'])  :
        data = []
        data.append(q)
        data.append(str(label))

        data_lst.append(data)
    return data_lst

In [ ]:
import sklearn
train_preprocessed=sklearn.utils.shuffle(train_preprocessed)

In [ ]:
train_preprocessed.head()

In [ ]:
# fold 개수
fold_num = 5

# Cross validation, StratifiedKfold
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=0)
folds=[]

# Unusual sentences must be included in the training data
# Seperate train set and validation set in each folds
for dataset_train, dataset_test in skf.split(train_preprocessed, train_preprocessed['encoded_label']):
  folds.append((dataset_train, dataset_test))

In [ ]:
for i in range(len(folds)):
  globals()['dataset_train_{}'.format(i)]= transform_df_lst(train_preprocessed.iloc[folds[i][0]])
  globals()['dataset_test_{}'.format(i)] = transform_df_lst(train_preprocessed.iloc[folds[i][1]])
  
dataset_list = [[dataset_train_0, dataset_test_0],[dataset_train_1, dataset_test_1],[dataset_train_2, dataset_test_2],
                [dataset_train_3, dataset_test_3],[dataset_train_4, dataset_test_4]]

KoBERT의 입력데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 셋팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

torch 형식의 데이터로 만들기

In [ ]:
#fold 별 학습데이터 리스트
fold_train_dataloader = []

#fold 별 검증데이터 리스트
fold_test_dataloader = []

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


for i in tqdm(range(5)):
   globals()['data_train_{}'.format(i)] = BERTDataset(globals()['dataset_train_{}'.format(i)], 0, 1, tok, max_len, True, False)
   globals()['data_test_{}'.format(i)] = BERTDataset(globals()['dataset_test_{}'.format(i)], 0, 1, tok, max_len, True, False)

#torch 형식의 데이터로 만들기
for i in tqdm(range(5)):
    globals()['train_dataloader_{}'.format(i)] = torch.utils.data.DataLoader(globals()['data_train_{}'.format(i)], batch_size=batch_size, num_workers=5)
    globals()['test_dataloader_{}'.format(i)] = torch.utils.data.DataLoader(globals()['data_test_{}'.format(i)], batch_size=batch_size, num_workers=5)

    fold_train_dataloader.append(globals()['data_train_{}'.format(i)])
    fold_test_dataloader.append(globals()['data_test_{}'.format(i)])

학습 및 검증 데이터 가져오기
- 선용 train_dataloader_0,train_dataloader_1번
- 유빈 train_dataloader_2,train_dataloader_3번
- 호재 train_dataloader_4번

KoBERT 학습모델 만들기
- 클래스 수 조정하기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 225 ,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

### KFold 때 마다 다르게 모델 선언

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.2).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
#loss_fn = FocalLoss() # focal loss ->
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader_1) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
#train_dataloader

데이터 학습하기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader_0)): # 여기 꼭 수정!!!
        # 돌릴 데이터가 fold 0 이면 train_dataloader_0 로 입력하기

        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader_0)): # 여기 꼭 수정!!!
        # 돌릴 데이터가 fold 0 이면 test_dataloader_0 로 입력하기

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    model_save_name = F'final_fold0_100_predict_epoch{e+1}.bert' # final_nodup_fold0_epoch 로 입력하기
    path = F"/content/MyDrive/MyDrive/test/{model_save_name}" # epoch 마다 저장
    torch.save(model.state_dict(), path)

## 동일 데이터에 대해서 5개 모델로 다르게 예측

테스트 데이터 읽어오기

In [ ]:
max_len = 64
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

new_test = nlp.data.TSVDataset('/content/MyDrive/MyDrive/test/test_preprocessed_ver2_no_lower_no_ques.tsv', field_indices=[4,5], num_discard_samples=1)

test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
predict_val = []
predict_num_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  predict_val.append(prediction)
  predict_num_lst.append(out.cpu().detach().numpy())
  if batch_id % 5000 == 0:
     print(str(batch_id) + "번째 문장의 분류 예측값은" + str(prediction) + "입니다.")

예측값 디코딩

In [ ]:
predict_df = test_preprocessed.copy()
predict_df["label"] = le.inverse_transform(predict_val)
predict_df.head()

저장형식 맞추기

In [ ]:
test_id = []
for i in range(1,100001):
    num = str(i).zfill(6)
    test_id.append("id_" + num)

In [ ]:
# label 분할
#predict_df["AI_id"] = test_id
predict_df["digit_1"] = predict_df["label"].apply(lambda x : x[0])
predict_df["digit_2"] = predict_df["label"].apply(lambda x : x[1:3] if len(x) == 6 else x[1]) # 4글자, 6글자인 경우 다르게 출력
predict_df["digit_3"] = predict_df["label"].apply(lambda x : x[3:] if len(x) == 6 else x[-2:]) # 4글자, 6글자인 경우 다르게 출력

In [ ]:
predict_df = predict_df[["AI_id","digit_1","digit_2","digit_3"]]
predict_df.head()

결과 저장하기

In [ ]:
np.save("/content/MyDrive/MyDrive/test/final_fold0_100_predict_epoch7.npy",predict_num_lst)